## RESNET IMPLEMENTATION

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Define the Gradient Boosting model
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
def evaluate(model, X, y):
    predictions = model.predict(X)
    accuracy = accuracy_score(y, predictions)
    precision = precision_score(y, predictions)
    recall = recall_score(y, predictions)
    f1 = f1_score(y, predictions)
    return accuracy, precision, recall, f1

# Evaluate on training data
train_accuracy, train_precision, train_recall, train_f1 = evaluate(model, X_train, y_train)
print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate(model, X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Train - Accuracy: 0.9528, Precision: 0.9498, Recall: 0.9659, F1: 0.9578
Test - Accuracy: 0.9507, Precision: 0.9491, Recall: 0.9649, F1: 0.9569


## SVM( LINEAR  ) IMPLEMENTATION

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Define the SVM model
model = SVC(kernel='linear', C=1, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
def evaluate(model, X, y):
    predictions = model.predict(X)
    accuracy = accuracy_score(y, predictions)
    precision = precision_score(y, predictions)
    recall = recall_score(y, predictions)
    f1 = f1_score(y, predictions)
    return accuracy, precision, recall, f1

# Evaluate on training data
train_accuracy, train_precision, train_recall, train_f1 = evaluate(model, X_train, y_train)
print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate(model, X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Train - Accuracy: 0.9284, Precision: 0.9263, Recall: 0.9461, F1: 0.9361
Test - Accuracy: 0.9285, Precision: 0.9262, Recall: 0.9498, F1: 0.9378


## CNN IMPLEMENTATION

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Add a channel dimension
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)  # Add a channel dimension
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * (X_train.shape[2] // 4), 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, define the loss function and the optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model():
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate_model(data, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(labels.cpu(), predicted.cpu())
        precision = precision_score(labels.cpu(), predicted.cpu())
        recall = recall_score(labels.cpu(), predicted.cpu())
        f1 = f1_score(labels.cpu(), predicted.cpu())
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 201):
    loss = train_model()
    if epoch % 10 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate_model(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Epoch: 010, Loss: 0.6136
Train - Accuracy: 0.7353, Precision: 0.6795, Recall: 0.9890, F1: 0.8055
Epoch: 020, Loss: 0.4629
Train - Accuracy: 0.8414, Precision: 0.8318, Recall: 0.8947, F1: 0.8621
Epoch: 030, Loss: 0.3162
Train - Accuracy: 0.8779, Precision: 0.8862, Recall: 0.8945, F1: 0.8904
Epoch: 040, Loss: 0.2525
Train - Accuracy: 0.8956, Precision: 0.9049, Recall: 0.9070, F1: 0.9060
Epoch: 050, Loss: 0.2166
Train - Accuracy: 0.9101, Precision: 0.9134, Recall: 0.9255, F1: 0.9194
Epoch: 060, Loss: 0.1950
Train - Accuracy: 0.9227, Precision: 0.9216, Recall: 0.9404, F1: 0.9309
Epoch: 070, Loss: 0.1819
Train - Accuracy: 0.9289, Precision: 0.9264, Recall: 0.9470, F1: 0.9365
Epoch: 080, Loss: 0.1720
Train - Accuracy: 0.9326, Precision: 0.9318, Recall: 0.9478, F1: 0.9397
Epoch: 090, Loss: 0.1640
Train - Accuracy: 0.9348, Precision: 0.9333, Recall: 0.9502, F1: 0.9417
Epoch: 100, Loss: 0.1572
Train - Accuracy: 0.9366, Precision: 0.9351, Recall: 0.9517, F1: 0.9433
Epoch: 110, Loss: 0.1509
Train

## CNN Leaky Relu

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Add a channel dimension
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)  # Add a channel dimension
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define the CNN model with LeakyReLU
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.01)
        self.fc1 = nn.Linear(32 * (X_train.shape[2] // 4), 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool(self.leaky_relu(self.conv1(x)))
        x = self.pool(self.leaky_relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, define the loss function and the optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model():
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate_model(data, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(labels.cpu(), predicted.cpu())
        precision = precision_score(labels.cpu(), predicted.cpu())
        recall = recall_score(labels.cpu(), predicted.cpu())
        f1 = f1_score(labels.cpu(), predicted.cpu())
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 201):
    loss = train_model()
    if epoch % 10 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate_model(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Epoch: 010, Loss: 0.6390
Train - Accuracy: 0.6203, Precision: 0.5936, Recall: 0.9986, F1: 0.7446
Epoch: 020, Loss: 0.5150
Train - Accuracy: 0.8217, Precision: 0.7955, Recall: 0.9131, F1: 0.8502
Epoch: 030, Loss: 0.3543
Train - Accuracy: 0.8786, Precision: 0.8741, Recall: 0.9123, F1: 0.8928
Epoch: 040, Loss: 0.2661
Train - Accuracy: 0.8950, Precision: 0.8974, Recall: 0.9151, F1: 0.9062
Epoch: 050, Loss: 0.2270
Train - Accuracy: 0.9101, Precision: 0.9138, Recall: 0.9251, F1: 0.9194
Epoch: 060, Loss: 0.2013
Train - Accuracy: 0.9198, Precision: 0.9159, Recall: 0.9419, F1: 0.9287
Epoch: 070, Loss: 0.1849
Train - Accuracy: 0.9267, Precision: 0.9225, Recall: 0.9474, F1: 0.9348
Epoch: 080, Loss: 0.1741
Train - Accuracy: 0.9288, Precision: 0.9265, Recall: 0.9466, F1: 0.9364
Epoch: 090, Loss: 0.1650
Train - Accuracy: 0.9310, Precision: 0.9289, Recall: 0.9482, F1: 0.9384
Epoch: 100, Loss: 0.1567
Train - Accuracy: 0.9336, Precision: 0.9319, Recall: 0.9496, F1: 0.9407
Epoch: 110, Loss: 0.1494
Train

## KNN Classifier

In [19]:
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train.numpy(), y_train.numpy())

# Make predictions on the train set
y_train_pred = knn.predict(X_train.numpy())

# Make predictions on the test set
y_test_pred = knn.predict(X_test.numpy())

# Evaluate the model on the training set
train_accuracy = accuracy_score(y_train.numpy(), y_train_pred)
train_precision = precision_score(y_train.numpy(), y_train_pred)
train_recall = recall_score(y_train.numpy(), y_train_pred)
train_f1 = f1_score(y_train.numpy(), y_train_pred)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test.numpy(), y_test_pred)
test_precision = precision_score(y_test.numpy(), y_test_pred)
test_recall = recall_score(y_test.numpy(), y_test_pred)
test_f1 = f1_score(y_test.numpy(), y_test_pred)

print(f'KNN Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')
print(f'KNN Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


KNN Train - Accuracy: 0.9665, Precision: 0.9658, Recall: 0.9741, F1: 0.9699
KNN Test - Accuracy: 0.9408, Precision: 0.9411, Recall: 0.9554, F1: 0.9482


## Logistic Regression

In [17]:
import torch.nn as nn
import torch.optim as optim

# Define the Logistic Regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 2)

    def forward(self, x):
        return self.linear(x)

# Instantiate the model, define the loss function and the optimizer
model = LogisticRegression(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model():
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate_model(data, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(labels.cpu(), predicted.cpu())
        precision = precision_score(labels.cpu(), predicted.cpu())
        recall = recall_score(labels.cpu(), predicted.cpu())
        f1 = f1_score(labels.cpu(), predicted.cpu())
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 201):
    loss = train_model()
    if epoch % 10 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate_model(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Epoch: 010, Loss: 0.8541
Train - Accuracy: 0.3405, Precision: 0.4075, Recall: 0.4186, F1: 0.4130
Epoch: 020, Loss: 0.8024
Train - Accuracy: 0.3823, Precision: 0.4450, Recall: 0.4629, F1: 0.4538
Epoch: 030, Loss: 0.7564
Train - Accuracy: 0.4602, Precision: 0.5121, Recall: 0.5510, F1: 0.5309
Epoch: 040, Loss: 0.7149
Train - Accuracy: 0.5286, Precision: 0.5669, Recall: 0.6336, F1: 0.5984
Epoch: 050, Loss: 0.6777
Train - Accuracy: 0.5932, Precision: 0.6171, Recall: 0.7007, F1: 0.6563
Epoch: 060, Loss: 0.6445
Train - Accuracy: 0.6451, Precision: 0.6581, Recall: 0.7487, F1: 0.7004
Epoch: 070, Loss: 0.6149
Train - Accuracy: 0.6825, Precision: 0.6876, Recall: 0.7829, F1: 0.7322
Epoch: 080, Loss: 0.5885
Train - Accuracy: 0.7159, Precision: 0.7125, Recall: 0.8170, F1: 0.7612
Epoch: 090, Loss: 0.5647
Train - Accuracy: 0.7417, Precision: 0.7367, Recall: 0.8311, F1: 0.7811
Epoch: 100, Loss: 0.5433
Train - Accuracy: 0.7655, Precision: 0.7583, Recall: 0.8468, F1: 0.8001
Epoch: 110, Loss: 0.5240
Train

## Naive Bayes

In [18]:
import torch
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Initialize and train the Naive Bayes classifier
nb = GaussianNB()
nb.fit(X_train.numpy(), y_train.numpy())

# Make predictions on the train set
y_train_pred = nb.predict(X_train.numpy())

# Make predictions on the test set
y_test_pred = nb.predict(X_test.numpy())

# Evaluate the model on the training set
train_accuracy = accuracy_score(y_train.numpy(), y_train_pred)
train_precision = precision_score(y_train.numpy(), y_train_pred)
train_recall = recall_score(y_train.numpy(), y_train_pred)
train_f1 = f1_score(y_train.numpy(), y_train_pred)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test.numpy(), y_test_pred)
test_precision = precision_score(y_test.numpy(), y_test_pred)
test_recall = recall_score(y_test.numpy(), y_test_pred)
test_f1 = f1_score(y_test.numpy(), y_test_pred)

print(f'Naive Bayes Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')
print(f'Naive Bayes Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Naive Bayes Train - Accuracy: 0.6030, Precision: 0.9957, Recall: 0.2850, F1: 0.4431
Naive Bayes Test - Accuracy: 0.5798, Precision: 0.9970, Recall: 0.2606, F1: 0.4131


## NAIVE BAYES

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define the ResNet block
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.BatchNorm1d(in_features),
            nn.ReLU(inplace=True),
            nn.Linear(in_features, in_features),
            nn.BatchNorm1d(in_features)
        )

    def forward(self, x):
        residual = x
        out = self.layer(x)
        out += residual
        out = F.relu(out)
        return out

# Define the ResNet model
class ResNet(nn.Module):
    def __init__(self, num_features, num_classes):
        super(ResNet, self).__init__()
        self.input_layer = nn.Linear(num_features, 64)
        self.block1 = ResidualBlock(64)
        self.block2 = ResidualBlock(64)
        self.block3 = ResidualBlock(64)
        self.output_layer = nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.input_layer(x))
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.output_layer(x)
        return F.log_softmax(x, dim=1)

# Model parameters
num_features = X_train.shape[1]
num_classes = 2  # Phishing or not

# Instantiate the model
model = ResNet(num_features, num_classes)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training the model
def train():
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate(data, labels):
    model.eval()
    with torch.no_grad():
        output = model(data)
        pred = output.argmax(dim=1)
        accuracy = accuracy_score(labels.cpu(), pred.cpu())
        precision = precision_score(labels.cpu(), pred.cpu())
        recall = recall_score(labels.cpu(), pred.cpu())
        f1 = f1_score(labels.cpu(), pred.cpu())
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 201):
    loss = train()
    if epoch % 10 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Epoch: 010, Loss: 0.1572
Train - Accuracy: 0.9268, Precision: 0.9122, Recall: 0.9604, F1: 0.9357
Epoch: 020, Loss: 0.1197
Train - Accuracy: 0.9439, Precision: 0.9311, Recall: 0.9706, F1: 0.9505
Epoch: 030, Loss: 0.0793
Train - Accuracy: 0.9638, Precision: 0.9528, Recall: 0.9835, F1: 0.9679
Epoch: 040, Loss: 0.0474
Train - Accuracy: 0.9729, Precision: 0.9594, Recall: 0.9931, F1: 0.9759
Epoch: 050, Loss: 0.0304
Train - Accuracy: 0.9773, Precision: 0.9633, Recall: 0.9969, F1: 0.9798
Epoch: 060, Loss: 0.0228
Train - Accuracy: 0.9885, Precision: 0.9831, Recall: 0.9963, F1: 0.9897
Epoch: 070, Loss: 0.0208
Train - Accuracy: 0.9895, Precision: 0.9906, Recall: 0.9904, F1: 0.9905
Epoch: 080, Loss: 0.0195
Train - Accuracy: 0.9902, Precision: 0.9869, Recall: 0.9955, F1: 0.9912
Epoch: 090, Loss: 0.0189
Train - Accuracy: 0.9905, Precision: 0.9884, Recall: 0.9945, F1: 0.9915
Epoch: 100, Loss: 0.0181
Train - Accuracy: 0.9902, Precision: 0.9892, Recall: 0.9931, F1: 0.9911
Epoch: 110, Loss: 0.0230
Train